In [ ]:
import pandas as pd
import random
import string

# 載入Excel檔案
file_path = '處理後的檔案.xlsx'
excel_file = pd.ExcelFile(file_path)

# 建立一個包含所有資料的 DataFrame
all_data = pd.concat([pd.read_excel(file_path, sheet_name=sheet) for sheet in excel_file.sheet_names])

while True:
    # 列出所有工作表並加入 "ALL" 選項
    print("可選擇的資料表如下：")
    for i, sheet_name in enumerate(excel_file.sheet_names):
        print(f"{i + 1}. {sheet_name}")
    print(f"{len(excel_file.sheet_names) + 1}. ALL")

    # 讓使用者選擇資料表
    sheet_index = int(input("請輸入想測驗的資料表序號：")) - 1
    if sheet_index == len(excel_file.sheet_names):
        df = all_data.copy()  # 選擇 "ALL" 選項時合併所有資料表
    else:
        df = pd.read_excel(file_path, sheet_name=excel_file.sheet_names[sheet_index])

    # 初始化分數和錯誤清單
    score = 0
    wrong_answers = []

    # 隨機打亂資料
    df = df.sample(frac=1).reset_index(drop=True)

    # 迭代每一行資料進行測驗
    for index, row in df.iterrows():
        explanation = row[1]  # 第二欄為解釋
        correct_answer = row[0]  # 第一欄為正確答案
        
        # 隨機選取幾個干擾選項
        options = [correct_answer]
        while len(options) < 5:  # 總共提供5個選項
            distractor = df.sample(1).iloc[0, 0]  # 隨機取另一個單字
            if distractor != correct_answer and distractor not in options:
                options.append(distractor)
        
        # 打亂選項順序
        random.shuffle(options)

        # 顯示選項
        print("\n解釋：", explanation)
        choices = dict(zip(string.ascii_uppercase[:len(options)], options))
        for letter, word in choices.items():
            print(f"{letter}. {word}")

        # 輸入選項
        user_choice = input("請輸入對應的選項（A, B, C, D, ...）： ").strip().upper()
        if user_choice in choices and choices[user_choice] == correct_answer:
            score += 1
            print("答對了！\n")
        else:
            print(f"答錯了! 正確答案是: {correct_answer}\n")
            wrong_answers.append({
                "解釋": explanation,
                "正確答案": correct_answer,
                "你的答案": choices.get(user_choice, "無效的選擇")
            })

    # 顯示總分
    print(f"測驗結束! 你的總分是: {score}/{len(df)}")

    # 列出錯誤題目
    if wrong_answers:
        print("\n以下是你答錯的題目：")
        for wrong in wrong_answers:
            print(f"解釋: {wrong['解釋']}")
            print(f"正確答案: {wrong['正確答案']}")
            print(f"你的答案: {wrong['你的答案']}\n")
    else:
        print("恭喜你，全都答對了!")

    # 提供再次測驗或退出選項
    again = input("是否要再測驗一次？輸入 'Q' 結束，或按任意鍵繼續： ")
    if again.strip().upper() == "Q":
        print("測驗結束，感謝參加！")
        break
